In [214]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import plotly.graph_objects as go
import sys
sys.path.append('../../assets/python/')
import tfb
import dmg5e
import estats5e

METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = True

In [215]:
# Import monster data
dfD = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','PB','XP','AC','HP Mean','AB','DPR Mean'
dfD['eHP Mean']  = dfD.apply(lambda row: estats5e.effHP(row['HP Mean'], row['AC'], 'linear'), axis=1)
dfD['eDPR Mean'] = dfD.apply(lambda row: estats5e.effDPR(row['DPR Mean'], row['AB'], 'linear'), axis=1)
dfD['eXP Mean']  = dfD.apply(lambda row: estats5e.effXP(row['HP Mean'], row['AC'], row['DPR Mean'], row['AB'], 'linear'), axis=1)
dfD.set_index('CR', inplace=True)

df0 = pd.read_csv('monster-cr-and-xp.csv')  # 'Book','Page','Type','Category','Legendary','CR','PB','XP',
                                            # 'HP','AC','adj HP','adj AC',
                                            # 'AB','adj AB','DPR', 'adj DPR'
df0 = df0.astype({'Book': 'category', 'Category': 'category', 'Legendary': 'category', 'Type': 'category'})

df0['eHP']  = df0.apply(lambda row: estats5e.effHP(row['adj HP'], row['adj AC'], 'linear'), axis=1)
df0['eDPR'] = df0.apply(lambda row: estats5e.effDPR(row['adj DPR'], row['adj AB'], 'linear'), axis=1)
df0['eXP']  = df0.apply(lambda row: estats5e.effXP(row['adj HP'], row['adj AC'], row['adj DPR'], row['adj AB'], 'linear'), axis=1)

df0['Legacy']  = df0.apply(lambda row: row['Category'] == 'legacy', axis=1)

df0['D-eCR']  = df0.apply(lambda row: float(np.interp(row['eHP'], dfD['eHP Mean'],  dfD.index, left=0, right=30)), axis=1)
df0['O-eCR']  = df0.apply(lambda row: float(np.interp(row['eDPR'], dfD['eDPR Mean'],  dfD.index, left=0, right=30)), axis=1)
df0['eCR']  = df0.apply(lambda row: float(np.interp(row['eXP'], dfD['eXP Mean'],  dfD.index, left=0, right=30)), axis=1)

df0['eHP Ratio'] = df0.apply(lambda row: row['eHP']/dfD.loc[row['CR'], 'eHP Mean'], axis=1)
df0['eDPR Ratio'] = df0.apply(lambda row: row['eDPR']/dfD.loc[row['CR'], 'eDPR Mean'], axis=1)
df0['eCR Delta'] = df0['eCR'] - df0['CR']
df0['eXP Ratio'] = df0['eXP'] / df0['XP']
df0['eXP Skew'] = (df0['eDPR Ratio'] - df0['eHP Ratio'])/(df0['eDPR Ratio'] + df0['eHP Ratio'])

df0['D-CR DMG'] = df0.apply(lambda row: dmg5e.monster_defensive_challenge_rating(row['adj HP'], row['adj AC']), axis=1)
df0['O-CR DMG'] = df0.apply(lambda row: dmg5e.monster_offensive_challenge_rating(row['adj DPR'], row['adj AB']), axis=1)
df0['C-CR DMG'] = df0.apply(lambda row: np.floor(dmg5e.monster_challenge_rating(row['adj HP'], row['adj AC'], row['adj DPR'], row['adj AB'])), axis=1)
df0['C-CR DMG Delta'] = df0['C-CR DMG'] - df0['CR']
df0['XP DMG'] = df0.apply(lambda row: dfD.loc[row['C-CR DMG'], 'XP'], axis=1)
df0['XP DMG Ratio'] = df0['XP DMG'] / df0['XP']
#df0['C-CR DMG'] = 0.5*(df0['D-CR DMG'] + df0['O-CR DMG'])
df0['CR spread'] = df0['CR'] + np.random.rand(len(df0.index.tolist())) - 0.5

print('Includes {} monsters in database'.format(len(df0.index.to_list())))

Includes 504 monsters in database


In [216]:
# Setup template and html config for plotly figures.
from scipy import optimize

def plot_mean_fit(fig, row, col, df0, xCol, yCol, name, line_color):
    # plot mean
    #dfM = df0[[xCol, yCol]].groupby(xCol).mean().reset_index()
    dfM = df0
    fig.add_trace(go.Scatter(
        x=dfM[xCol], y=dfM[yCol],
        mode='markers', 
        name=f'{name}',
        legendgroup=name, 
        line_color=line_color,
        hovertemplate='<b>' + name + '</b><br>' + 'CR %{x:,.0f}<br>' + 'Mean %{y:,.1f}<br>' + '<extra></extra>'
    ), row=row, col=col)
    coefs = np.polyfit(dfM[xCol], dfM[yCol], 1)
    poly = np.poly1d(coefs)
    fig.add_trace(go.Scatter(
        x=dfM[xCol], y=poly(dfM[xCol]),
        mode='lines', 
        name=f'{name}',
        legendgroup=name, 
        line_color=line_color,
        hoverinfo='skip',
        showlegend=False,
    ), row=row, col=col)

def plot_confidence_interval(fig, df0, conf, xCol, yCol, name, line_color, fillcolor):
    # plot confidence
    if conf > 0:
        dfU = df0[[xCol, yCol]].groupby(xCol).quantile(0.5 + conf/2).reset_index()
        dfL = df0[[xCol, yCol]].groupby(xCol).quantile(0.5 - conf/2).reset_index()
        fig.add_trace(go.Scatter(x=dfU[xCol], y=dfU[yCol], line_width=0, legendgroup=name, showlegend=False, hoverinfo='skip'))
        fig.add_trace(go.Scatter(x=dfL[xCol], y=dfL[yCol], line_width=0, name=f'{name} - {conf:.0%} confidence', legendgroup=name, hoverinfo='skip', fill='tonexty', fillcolor=fillcolor))
    
    # plot mean
    dfM = df0[[xCol, yCol]].groupby(xCol).mean().reset_index()
    fig.add_trace(go.Scatter(
        x=dfM[xCol], y=dfM[yCol],
        mode='lines', 
        name=f'{name} - mean',
        legendgroup=name, 
        line_color=line_color,
        hovertemplate='<b>' + name + '</b><br>' + 'CR %{x:,.0f}<br>' + 'Mean %{y:,.2f}<br>' + '<extra></extra>'
    ))

def piecewise_linear(x, x0, y0, k1, k2):
    #return np.piecewise(x, [x < x0], [lambda x:k1*x + y0-k1*x0, lambda x:k2*x + y0-k2*x0])
    return np.piecewise(x, [x < 20], [lambda x:k1*x + y0-k1*20, lambda x:k2*x + y0-k2*20])

def plot_data_and_piecewise_fit(fig, x, y, **kwargs):
    row = kwargs.pop('row') if 'row' in kwargs else 1
    col = kwargs.pop('col') if 'col' in kwargs else 1
    
    # plot data
    fig.add_trace(go.Scatter(
        x=x, y=y,
        mode='markers', 
        **kwargs,
    ), row=row, col=col)

    # plot fit
    kwargs['hoverinfo'] = 'skip'
    kwargs['showlegend'] = False
    kwargs['line_dash'] = 'solid'
    p, e = optimize.curve_fit(piecewise_linear, x, y)
    xr = np.linspace(1,30,30)
    fig.add_trace(go.Scatter(
        x=xr, 
        y=piecewise_linear(xr, *p),
        mode='lines', 
        **kwargs,
    ), row=row, col=col)
    return fig, piecewise_linear(xr, *p)
    #return fig

In [217]:
# plot the number of monsters by the type of change
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        barmode='stack',
        #xaxis=dict(title_text='change'),
        yaxis=dict(
            title_text='monsters changed (%)',
            #range=[0,100],
            tick0=0, dtick=10,
            minor=dict(tick0=0, dtick=5),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
            traceorder='normal',
        )
    )
)

df1 = df0[df0['Book'].isin(['MPMotM','VGtM','MToF'])]
dfMP = pd.pivot_table(df1, 
    values=['HP','AC','AB','DPR',
            'adj HP','adj AC','adj AB','adj DPR',
            'eHP','eDPR','eXP',
            'eHP Ratio','eDPR Ratio','eXP Ratio','eXP Skew'], 
    index=['Monster', 'Legendary'], columns=['Legacy']).reset_index()
m_count = dfMP.shape[0]
print(m_count)

stats = ['HP', 'AC', 'DPR', 'AB']
data = {}
for stat in stats:
    data[stat] = np.sum((dfMP[stat][False] - dfMP[stat][True]) != 0)

fig.add_trace(go.Bar(
    x=np.array(list(data.keys())),
    y=np.array(list(data.values()))*(100/m_count),
    name='Raw stats',
    hovertemplate = ''
        + 'Raw %{y:,.1f}%'
        + '<extra></extra>'
))
print(data)

stats = ['HP', 'AC', 'DPR', 'AB']
for stat in stats:
    data[stat] = np.sum((dfMP[f'adj {stat}'][False] - dfMP[f'adj {stat}'][True]) != 0) - data[stat]

fig.add_trace(go.Bar(
    x=np.array(list(data.keys())),
    y=np.array(list(data.values()))*(100/m_count),
    name='Adjusted stats',
    hovertemplate = ''
        + 'Adjusted %{y:,.1f}%'
        + '<extra></extra>'
))
print(data)

# show figure
fig.update_layout(width=600, height=450, hovermode='x unified')
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-stat-changes-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-stat-changes-small')

252
{'HP': 61, 'AC': 10, 'DPR': 130, 'AB': 14}
{'HP': 21, 'AC': 28, 'DPR': 8, 'AB': 7}


In [218]:
# plot the number of monsters changed by CR
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        barmode='stack',
        xaxis=dict(
            title_text='challenge rating',
            range=[0, 31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='monsters',
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=1.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
            traceorder='normal',
        )
    )
)

def x_decoder(x):
    match x:
        case 0:
            return 0
        case 0.125:
            return 0.125
        case 0.25:
            return 0.28125
        case 0.5:
            return 0.5625
        case 1:
            return 1.125
        case _:
            return x

def w_decoder(x):
    match x:
        case 0:
            return 0.125
        case 0.125:
            return 0.125
        case 0.25:
            return 0.1875
        case 0.5:
            return 0.375
        case 1:
            return 0.75
        case _:
            return 1.0


dfMP = pd.pivot_table(
    df0[df0['Book'].isin(['MPMotM','VGtM','MToF'])], 
    values=['HP','AC','AB','DPR',
            'adj HP','adj AC','adj AB','adj DPR',
            'eHP','eDPR','eXP',
            'eHP Ratio','eDPR Ratio','eXP Ratio','eXP Skew'], 
    index=['Monster', 'Legendary','CR'], columns=['Legacy']).reset_index()
dfMP['Status'] = dfMP.apply(lambda row: 'Changed' if row['eXP'][True] != row['eXP'][False] else 'Unchanged', axis=1)
dfG = dfMP[['Monster','Status']].groupby(['Status']).count()

for g in ['Unchanged','Changed']:
    dfC = dfMP[['CR','Status']]
    dfC = dfC[dfC['Status'].isin([g])].groupby(['CR']).count()
    x = [x_decoder(cr) for cr in dfC.index]
    w = [w_decoder(cr) for cr in dfC.index]
    cr = dfC.index.to_list()
        
    fig.add_trace(go.Bar(
        x=x,
        y=dfC['Status'],
        width=w,
        name=g,
        customdata=dfC.index,
        hovertemplate = '<b>' + g + '</b><br>'
            + 'CR %{customdata}<br>'
            + 'Monsters %{y:,.0f}'
            + '<extra></extra>'
    ))


# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-change-distribution-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-change-distribution-small')

In [230]:
# plots XP Ratio as a function of CR
columns = ['eXP Ratio']

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='challenge rating',
            range=[0, 26],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='calculated XP ratio',
            range=[0, 2.5],
            tick0=0, dtick=0.5,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
            bgcolor='rgba(0,0,0,0)',
        )
    )
)

# plot data
df1 = df0[df0['Book'].isin(['MPMotM','VGtM','MToF']) & df0['Legacy'].eq(True)]
plot_confidence_interval(fig, df1, 0.8, 'CR', 'eXP Ratio', 'MToF + VGtM', 'rgba(0, 0, 255, 1.0)', 'rgba(0, 0, 255, 0.3)')

df1 = df0[df0['Book'].isin(['MPMotM','VGtM','MToF']) & df0['Legacy'].eq(False)]
plot_confidence_interval(fig, df1, 0.8, 'CR', 'eXP Ratio', 'MotM', 'rgba(255, 0, 0, 1.0)', 'rgba(255, 0, 0, 0.3)')


# show figure
fig.update_layout(width=600, height=500)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-xp-ratio-vs-cr-large')
    tfb.save_fig_html(fig, format='small', name='fig-xp-ratio-vs-cr-small')

In [220]:
# plots the change in XP vs incoming XP ratio
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='$XP / XP_{\mathrm{CR}}$',
            autorange=True,
            #range=[-0.5, 0.5],
            tick0=0, dtick=0.50,
            minor=dict(tick0=0, dtick=0.10),
        ),
        yaxis=dict(
            title_text='$XP^{\,\prime} / XP$',
            #autorange=True,
            range=[0, 2.5],
            #tick0=0, dtick=0.5,
            #minor=dict(tick0=0, dtick=0.25),
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=1.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)


df1 = df0[df0['Book'].isin(['MPMotM','VGtM','MToF'])]
dfMP = pd.pivot_table(df1, 
    values=['HP','AC','AB','DPR',
            'adj HP','adj AC','adj AB','adj DPR',
            'eHP','eDPR','eXP',
            'eHP Ratio','eDPR Ratio','eXP Ratio','eXP Skew'], 
    index=['Monster', 'Legendary', 'CR'], columns=['Legacy']).reset_index()
dfMP['Changed'] = dfMP['eXP'][True] != dfMP['eXP'][False]
dfMP['eXP ratio delta'] = dfMP['eXP'][False] / dfMP['eXP'][True]
dfMP = dfMP[dfMP['Changed']]

cr_ranges = [(0,5),(6,30)]
for cr_min, cr_max in cr_ranges:
    dft = dfMP[dfMP['CR'].between(cr_min, cr_max)]
    fig.add_trace(go.Scatter(
        x=dft['eXP Ratio'][True], 
        y=dft['eXP ratio delta'], 
        customdata=np.stack((dft['Monster'], dft['CR']), axis=-1),
        mode='markers', 
        name=f'CR {cr_min} - {cr_max}',
        showlegend=True,
        hovertemplate=
            '<b>%{customdata[0]}</b><br>'+
            'CR %{customdata[1]}<br>'+
            'XP ratio %{x:,.2f}<br>'+
            'XP delta %{y:,.2f}' + 
            '<extra></extra>'
    ))

fig.add_trace(go.Scatter(
    x=np.linspace(0.2,2,40), 
    y=1/np.linspace(0.2,2,40), 
    mode='lines', 
    name='changed defense',
    showlegend=False,
    hoverinfo='skip',
    line=dict(color='black',dash='dash'),
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-xp-delta-vs-xp-ratio-large')
    tfb.save_fig_html(fig, format='small', name='fig-xp-delta-vs-xp-ratio-small')

In [221]:
# plot damage breakdown by type for original and republished monsters
import plotly.graph_objects as go
import numpy as np

damage_type_names = [
    'acid', 'cold',  'fire', 'force', 'lightning','necrotic', 'poison', 
    'psychic', 'radiant', 'thunder', 'bludgeoning', 'piercing', 'slashing',
]
cr_min = 1
cr_max = 30
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=60, r=25, b=70, t=20, pad=4),
        #xaxis=dict(title_text='damage type'),
        yaxis=dict(
            title_text='damage (%)',
            tickformat=',.0%',
            automargin=True,
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

df = pd.read_csv('./monster-parsed-damage.csv')
dfG = df[df['version'].isin(['old']) & df['CR'].between(cr_min,cr_max)][damage_type_names].sum()
dfG['total'] = dfG['acid'] + dfG['cold'] + dfG['fire'] + dfG['force'] + dfG['lightning'] + dfG['necrotic'] + dfG['poison'] + dfG['psychic'] + dfG['radiant'] + dfG['thunder'] + dfG['bludgeoning'] + dfG['piercing'] + dfG['slashing']
fig.add_trace(go.Bar(
    x=damage_type_names,
    y=dfG[damage_type_names].to_numpy()/dfG['total'],
    name='VGtM + MToF',
    hovertemplate='<b>%{x}</b><br>' + 'Damage %{y:,.1%}<br>' + '<extra></extra>',
))

dfG = df[df['version'].isin(['new']) & df['CR'].between(cr_min,cr_max)][damage_type_names].sum()
dfG['total'] = dfG['acid'] + dfG['cold'] + dfG['fire'] + dfG['force'] + dfG['lightning'] + dfG['necrotic'] + dfG['poison'] + dfG['psychic'] + dfG['radiant'] + dfG['thunder'] + dfG['bludgeoning'] + dfG['piercing'] + dfG['slashing']
fig.add_trace(go.Bar(
    x=damage_type_names,
    y=dfG[damage_type_names].to_numpy()/dfG['total'],
    name='MotM',
    hovertemplate='<b>%{x}</b><br>' + 'Damage %{y:,.1%}<br>' + '<extra></extra>',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-damage-by-type-large')
    tfb.save_fig_html(fig, format='small', name='fig-damage-by-type-small')

In [222]:
# Plots the number of monsters who changed for each type of condition immunity
df1 = df0[df0['CR'].between(1/8, 30)]

CONDITIONS = [
    'Blinded','Charmed','Deafened','Exhaustion','Frightened',
    'Grappled','Incapacitated','Invisible','Paralyzed','Petrified',
    'Poisoned','Prone','Restrained','Stunned','Unconscious']

"""# create custom text
total_monsters = df1.shape[0]
total_immune = [int(df1[c].sum().tolist()) for c in conditions]
pct_immune = [100*df1[c].mean().tolist() for c in conditions]
custom_text = [f'{p:.0f}% ({i} out of {total_monsters})' for p, i in zip(pct_immune, total_immune)]"""

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=60, r=25, b=120, t=20, pad=4),
        xaxis=dict(
            autorange=True,
            #range=[-0.5, len(CONDITIONS)-0.5],
        ),
        yaxis=dict(
            title_text='percent immune',
            range=[-0.1, 10],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        )
    )
)

df1 = df0[df0['Book'].isin(['MPMotM','VGtM','MToF'])]
dfMP = pd.pivot_table(df1, 
    values=CONDITIONS, 
    index=['Monster', 'Legendary'], columns=['Legacy']).reset_index()
num_monsters = len(dfMP['Monster'])

counts = []
for c in CONDITIONS:
    indx = dfMP[c][False] != dfMP[c][True]
    counts.append(np.sum(indx))
    print(c, '-', dfMP['Monster'][indx].to_list())

fig.add_trace(go.Scatter(
    x=CONDITIONS,
    y=counts,
    mode='markers', 
    marker=dict(size=12, line=dict(width=1, color='Black')),
    #text=custom_text,
    #unselected=dict(marker=dict(opacity=0.0)),
    hovertemplate=
        '<b>Condition:</b> %{x}<br>'+
        '<b>Count</b> %{y:.0f}'+
        '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=500)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name='fig-xp-ratio-vs-cr-large')
    #tfb.save_fig_html(fig, format='small', name='fig-xp-ratio-vs-cr-small')

Blinded - ['Molydeus']
Charmed - ['Yuan-ti Broodguard']
Deafened - ['Molydeus']
Exhaustion - ['Molydeus']
Frightened - ['Abishai, Red']
Grappled - []
Incapacitated - []
Invisible - []
Paralyzed - ['Yuan-ti Broodguard']
Petrified - []
Poisoned - ['Corpse Flower']
Prone - []
Restrained - []
Stunned - []
Unconscious - []


In [223]:
# Plots the number of monsters who changed for each type of damage immunity
df1 = df0[df0['CR'].between(1/8, 30)]

DAMAGE_TYPES = [
    'Bludgeoning','Piercing','Slashing',
    'Acid','Cold','Fire','Force','Lightning',
    'Necrotic','Poison','Psychic','Radiant','Thunder']

"""# create custom text
total_monsters = df1.shape[0]
total_immune = [int(df1[c].sum().tolist()) for c in conditions]
pct_immune = [100*df1[c].mean().tolist() for c in conditions]
custom_text = [f'{p:.0f}% ({i} out of {total_monsters})' for p, i in zip(pct_immune, total_immune)]"""

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=60, r=25, b=120, t=20, pad=4),
        xaxis=dict(
            autorange=True,
            #range=[-0.5, len(CONDITIONS)-0.5],
        ),
        yaxis=dict(
            title_text='percent immune',
            range=[-0.5, 20],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        )
    )
)

df1 = df0[df0['Book'].isin(['MPMotM','VGtM','MToF'])]
dfMP = pd.pivot_table(df1, 
    values=['HP','adj HP','eHP'] + DAMAGE_TYPES, 
    index=['Monster', 'Legendary'], columns=['Legacy']).reset_index()
num_monsters = len(dfMP['Monster'])

counts = []
monsters = []
for c in DAMAGE_TYPES:
    indx = dfMP[c][False] != dfMP[c][True]
    counts.append(np.sum(indx))
    delta = np.mean(dfMP['eHP'][False][indx] / dfMP['eHP'][True][indx] - 1)
    print(f'{c} - ({delta:.4f})', dfMP['Monster'][indx].to_list())
    monsters += dfMP['Monster'][indx].to_list()

monsters = list(set(monsters))
print(sorted(monsters))

fig.add_trace(go.Scatter(
    x=DAMAGE_TYPES,
    y=counts,
    mode='markers', 
    marker=dict(size=12, line=dict(width=1, color='Black')),
    #text=custom_text,
    #unselected=dict(marker=dict(opacity=0.0)),
    hovertemplate=
        '<b>Condition:</b> %{x}<br>'+
        '<b>Count</b> %{y:.0f}'+
        '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=500)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name='fig-xp-ratio-vs-cr-large')
    #tfb.save_fig_html(fig, format='small', name='fig-xp-ratio-vs-cr-small')

Bludgeoning - (0.0023) ['Alhoon', 'Annis Hag', 'Boneclaw', 'Clockwork Bronze Scout', 'Clockwork Oaken Bolter', 'Clockwork Stone Defender', 'Eladrin, Autumn', 'Eladrin, Spring', 'Eladrin, Summer', 'Eladrin, Winter', 'Morkoth', 'Nupperibo', 'Shoosuva', 'Slithering Tracker', 'Troll, Spirit', 'Warlock of the Fiend']
Piercing - (0.0023) ['Alhoon', 'Annis Hag', 'Boneclaw', 'Clockwork Bronze Scout', 'Clockwork Oaken Bolter', 'Clockwork Stone Defender', 'Eladrin, Autumn', 'Eladrin, Spring', 'Eladrin, Summer', 'Eladrin, Winter', 'Morkoth', 'Nupperibo', 'Shoosuva', 'Slithering Tracker', 'Troll, Spirit', 'Warlock of the Fiend']
Slashing - (0.0023) ['Alhoon', 'Annis Hag', 'Boneclaw', 'Clockwork Bronze Scout', 'Clockwork Oaken Bolter', 'Clockwork Stone Defender', 'Eladrin, Autumn', 'Eladrin, Spring', 'Eladrin, Summer', 'Eladrin, Winter', 'Morkoth', 'Nupperibo', 'Shoosuva', 'Slithering Tracker', 'Troll, Spirit', 'Warlock of the Fiend']
Acid - (0.0000) ['Juiblex']
Cold - (nan) []
Fire - (-0.3155) ['B